# 5. Kafka Consumer - Read Messages from sample-topic

Consumes messages from the `sample-topic` produced by the Kafka Producer (notebook 4).

**Run location:** This notebook must run from **inside the VPC** (e.g., SageMaker notebook instance). MSK brokers use private DNS and are unreachable from your laptop.

In [ ]:
import json
import boto3
from kafka import KafkaConsumer

In [ ]:
# Get Kafka bootstrap servers from SSM (same as producer)
ssm = boto3.client('ssm')
bootstrap_servers_raw = ssm.get_parameter(Name='/kafka/bootstrap_servers', WithDecryption=True)['Parameter']['Value']
bootstrap_servers = [s.strip() for s in bootstrap_servers_raw.split(',')]
print(f"Bootstrap servers: {bootstrap_servers}")

In [ ]:
TOPIC = 'sample-topic'

# Create consumer (reads from earliest, stops after 5s of no new messages)
consumer = KafkaConsumer(
    TOPIC,
    bootstrap_servers=bootstrap_servers,
    auto_offset_reset='earliest',
    consumer_timeout_ms=5000,
    request_timeout_ms=15000,
    api_version_auto_timeout_ms=15000,
    value_deserializer=lambda m: json.loads(m.decode('utf-8')),
    key_deserializer=lambda k: k.decode('utf-8') if k else None,
)

## Consume and print messages

In [ ]:
print(f"Consuming from topic '{TOPIC}'...\n")
messages = list(consumer)
print(f"Received {len(messages)} messages:\n")
for msg in messages:
    print(f"  partition={msg.partition} offset={msg.offset} key={msg.key}")
    print(f"  value: {msg.value}\n")
consumer.close()

## Optional: Consume with continuous polling

In [ ]:
# Re-create consumer for continuous polling (interrupt to stop)
consumer = KafkaConsumer(
    TOPIC,
    bootstrap_servers=bootstrap_servers,
    auto_offset_reset='latest',
    consumer_timeout_ms=5000,
    request_timeout_ms=15000,
    value_deserializer=lambda m: json.loads(m.decode('utf-8')),
    key_deserializer=lambda k: k.decode('utf-8') if k else None,
)
for msg in consumer:
    print(msg.value)
consumer.close()